In [19]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm import tqdm

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error

pd.set_option('display.max_columns', 2000)


In [2]:
df_train = pd.read_pickle('../data/train/full_comp.pickle')
#df_test = pd.read_pickle('../data/test/full.pickle')


In [3]:
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_l

In [4]:
df_train = pd.merge(df_train, pd.read_pickle('../data/train/roberta_vec_sample_doi_cites.pickle'), on='id', how='left')

In [5]:
df_pre_train = df_train[df_train['cites'].isnull()].reset_index(drop=True)
df_pre_valid = df_train[df_train['cites'].isnull()==False].reset_index(drop=True)
df_pre_train.shape, df_pre_valid.shape



((836407, 1256), (15117, 1256))

In [6]:
del_col = []
for col in df_train.columns:
    if 'doi_cites' in col:
        del_col.append(col)
del_col

['doi_cites',
 'doi_cites_mean_author_first_label',
 'doi_cites_count_author_first_label',
 'doi_cites_sum_author_first_label',
 'doi_cites_min_author_first_label',
 'doi_cites_max_author_first_label',
 'doi_cites_median_author_first_label',
 'doi_cites_std_author_first_label',
 'doi_cites_q10_author_first_label',
 'doi_cites_q25_author_first_label',
 'doi_cites_q75_author_first_label',
 'doi_cites_mean_doi_id_label',
 'doi_cites_count_doi_id_label',
 'doi_cites_sum_doi_id_label',
 'doi_cites_min_doi_id_label',
 'doi_cites_max_doi_id_label',
 'doi_cites_median_doi_id_label',
 'doi_cites_std_doi_id_label',
 'doi_cites_q10_doi_id_label',
 'doi_cites_q25_doi_id_label',
 'doi_cites_q75_doi_id_label',
 'doi_cites_mean_pub_publisher_label',
 'doi_cites_count_pub_publisher_label',
 'doi_cites_sum_pub_publisher_label',
 'doi_cites_min_pub_publisher_label',
 'doi_cites_max_pub_publisher_label',
 'doi_cites_median_pub_publisher_label',
 'doi_cites_std_pub_publisher_label',
 'doi_cites_q10_pub_pu

In [7]:
SEED = 777
# --------------------------------------
# パラメータ定義
# --------------------------------------
lgb_params = {
    'objective': 'root_mean_squared_error',
    'boosting_type': 'gbdt',
    'n_estimators': 2000,
    #'colsample_bytree': 0.8,
    #'subsamples': 0.8,
    #'bagging_freq': 1,
    #'n_estimators': 50000,
    #'learning_rate': 0.01,
}



# 目的変数
target = 'doi_cites'

y_train = df_pre_train[target].values
y_valid = df_pre_valid[target].values

x_train = df_pre_train.copy()
x_valid = df_pre_valid.copy()

x_train = x_train.drop(
    ['id', 'authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'update_date_x', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
    , axis=1
)

x_valid = x_valid.drop(
    ['id', 'authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'update_date_x', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
    , axis=1
)

# drop
x_train = x_train.drop(del_col, axis=1)
x_valid = x_valid.drop(del_col, axis=1)

# LightGBM
model = lgb.LGBMRegressor(**lgb_params)
model.fit(x_train, y_train,
            eval_set=(x_valid, y_valid),
            eval_metric='mse',
            verbose=100,
            early_stopping_rounds=20,
)

pred_doi_cites = model.predict(x_valid)
rmsle = mean_squared_error(y_valid, pred_doi_cites, squared=False)
print(f"rmsle: {rmsle}")


/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 20 rounds
[100]	valid_0's l2: 0.949298	valid_0's rmse: 0.974319
[200]	valid_0's l2: 0.926572	valid_0's rmse: 0.962586
[300]	valid_0's l2: 0.917492	valid_0's rmse: 0.957858
[400]	valid_0's l2: 0.91167	valid_0's rmse: 0.954814
[500]	valid_0's l2: 0.908383	valid_0's rmse: 0.953091
Early stopping, best iteration is:
[529]	valid_0's l2: 0.907279	valid_0's rmse: 0.952512
rmsle: 0.9525120447140948


In [8]:
x_train.head()

,pub_journals,pub_dois,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_num,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,is_null_comments,is_null_report-no,is_null_journal-ref,rbert_vec_raw_768_0,rbert_vec_raw_768_1,rbert_vec_raw_768_2,rbert_vec_raw_768_3,rbert_vec_raw_768_4,rbert_vec_raw_768_5,rbert_vec_raw_768_6,rbert_vec_raw_768_7,rbert_vec_raw_768_8,rbert_vec_raw_768_9,rbert_vec_raw_768_10,rbert_vec_raw_768_11,rbert_vec_raw_768_12,rbert_vec_raw_768_13,rbert_vec_raw_768_14,rbert_vec_raw_768_15,rbert_vec_raw_768_16,rbert_vec_raw_768_17,rbert_vec_raw_768_18,rbert_vec_raw_768_19,rbert_vec_raw_768_20,rbert_vec_raw_768_21,rbert_vec_raw_768_22,rbert_vec_raw_768_23,rbert_vec_raw_768_24,rbert_vec_raw_768_25,rbert_vec_raw_768_26,rbert_vec_raw_768_27,rbert_vec_raw_768_28,rbert_vec_raw_768_29,rbert_vec_raw_768_30,rbert_vec_raw_768_31,rbert_vec_raw_768_32,rbert_vec_raw_768_33,rbert_vec_raw_768_34,rbert_vec_raw_768_35,rbert_vec_raw_768_36,rbert_vec_raw_768_37,rbert_vec_raw_768_38,rbert_vec_raw_768_39,rbert_vec_raw_

In [9]:
feature = x_train.copy()
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(20)


,importance
update_date_unixtime,1.311019e+06
doi_id_label,8.177456e+05
update_ym,5.606022e+05
author_first_label,5.085992e+05
submitter_label,4.781559e+05
pub_publisher_label,4.730973e+05
author_num,2.478387e+05
category_name_label,1.715062e+05
first_created_days,1.439122e+05
category_name_parent_label,1.239431e+05


In [10]:
df_train = df_train[df_train['cites'].isnull() == False].reset_index(drop=True)
df_train['cites'] = np.log1p(df_train['cites'])
df_train['pred_doi_cites'] = pred_doi_cites
df_train.shape

(15117, 1257)

In [11]:
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_l

In [12]:
NFOLDS = 10
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold
df_train.head()


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_l

In [13]:
def make_statics_table(target, key, x_train, x_valid, x_test):
    target_encoding = pd.concat([
        pd.DataFrame(x_train.groupby(key)[target].count()).rename(columns={target:f'te_{key}_{target}_count'}),
        pd.DataFrame(x_train.groupby(key)[target].min()).rename(columns={target:f'te_{key}_{target}_min'}),
        pd.DataFrame(x_train.groupby(key)[target].max()).rename(columns={target:f'te_{key}_{target}_max'}),
        pd.DataFrame(x_train.groupby(key)[target].std()).rename(columns={target:f'te_{key}_{target}_std'}),
        pd.DataFrame(x_train.groupby(key)[target].mean()).rename(columns={target:f'te_{key}_{target}_mean'}),
        pd.DataFrame(x_train.groupby(key)[target].median()).rename(columns={target:f'te_{key}_{target}_median'}),
    ], axis = 1)

    x_train = pd.merge(x_train, target_encoding, on=key, how='left')
    x_valid = pd.merge(x_valid, target_encoding, on=key, how='left')
    x_test = pd.merge(x_test, target_encoding, on=key, how='left')

    x_train[f'te_diff_{key}_{target}_mean'] = x_train['doi_cites'] - x_train[f'te_{key}_{target}_mean']
    x_train[f'te_rate_{key}_{target}_mean'] = x_train['doi_cites'] / x_train[f'te_{key}_{target}_mean']
    x_valid[f'te_diff_{key}_{target}_mean'] = x_valid['doi_cites'] - x_valid[f'te_{key}_{target}_mean']
    x_valid[f'te_rate_{key}_{target}_mean'] = x_valid['doi_cites'] / x_valid[f'te_{key}_{target}_mean']
    x_test[f'te_diff_{key}_{target}_mean'] = x_test['doi_cites'] - x_test[f'te_{key}_{target}_mean']
    x_test[f'te_rate_{key}_{target}_mean'] = x_test['doi_cites'] / x_test[f'te_{key}_{target}_mean']

    return x_train, x_valid, x_test

In [16]:
np.where(x_train.dtypes == 'category')[0]

array([304, 305, 306, 307, 308, 309, 311, 312, 313])

In [31]:
SEED = 777
# --------------------------------------
# パラメータ定義
# --------------------------------------
lgb_params = {
    'objective': 'root_mean_squared_error',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    #'colsample_bytree': 0.8,
    #'subsamples': 0.8,
    #'bagging_freq': 1,
    #'n_estimators': 50000,
    #'learning_rate': 0.01,
}

result_y = []
result_proba = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'update_date_x', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    # target encoding
    #target = 'doi_cites'
    #key = 'pred_doi_cites'
    #x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)

    #keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label', 'category_name_label', 'pred_doi_cites']
    '''
    keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)
    '''
    
    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='mse',
                verbose=100,
                early_stopping_rounds=20,
    )

    fold_result = model.predict(x_test)
    result_y.extend(y_test)
    result_proba.extend(fold_result)
    rmsle = mean_squared_error(y_test, fold_result, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")

rmsle = mean_squared_error(result_y, result_proba, squared=False)
print("+-" * 40)
print(f"score: {rmsle}")


/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[75]	valid_0's l2: 0.234876	valid_0's rmse: 0.484641
fold 0 lgb score: 0.4904175791010282
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't

In [33]:
SEED = 777
result_y = []
result_proba = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'update_date_x', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    # target encoding
    #target = 'doi_cites'
    #key = 'pred_doi_cites'
    #x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)

    #keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label', 'category_name_label', 'pred_doi_cites']
    '''
    keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)
    '''
    
    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)


    cat_features = np.where(x_train.dtypes == 'category')[0]
    train_pool = Pool(x_train, y_train, cat_features=cat_features)
    validate_pool = Pool(x_valid, y_valid, cat_features=cat_features)
    test_pool = Pool(x_test, y_test, cat_features=cat_features)
    
    model = CatBoostRegressor(loss_function='RMSE')
    #model = lgb.LGBMRegressor(**lgb_params)
    model.fit(train_pool,
                eval_set=validate_pool,
                verbose=100,
                early_stopping_rounds=20,
    )

    fold_result = model.predict(x_test)
    result_y.extend(y_test)
    result_proba.extend(fold_result)
    rmsle = mean_squared_error(y_test, fold_result, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")

rmsle = mean_squared_error(result_y, result_proba, squared=False)
print("+-" * 40)
print(f"score: {rmsle}")


Learning rate set to 0.073805
0:	learn: 1.1320861	test: 1.1292688	best: 1.1292688 (0)	total: 47.6ms	remaining: 47.6s
100:	learn: 0.4906269	test: 0.4917286	best: 0.4917286 (100)	total: 4.94s	remaining: 44s
200:	learn: 0.4557686	test: 0.4857646	best: 0.4852757 (187)	total: 9.7s	remaining: 38.6s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.4852756951
bestIteration = 187

Shrink model to first 188 iterations.
fold 0 lgb score: 0.48819686630329834
Learning rate set to 0.073805
0:	learn: 1.1347040	test: 1.1101022	best: 1.1101022 (0)	total: 53.3ms	remaining: 53.2s
100:	learn: 0.4855378	test: 0.5231290	best: 0.5231290 (100)	total: 4.97s	remaining: 44.3s
200:	learn: 0.4543426	test: 0.5203453	best: 0.5203093 (189)	total: 9.93s	remaining: 39.5s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.519824723
bestIteration = 204

Shrink model to first 205 iterations.
fold 1 lgb score: 0.483903008045062
Learning rate set to 0.073805
0:	learn: 1.1333428	test: 1.14

In [28]:
x_train.columns[categorical_features_indices]

Index(['submitter_label', 'doi_id_label', 'author_first_label',
       'pub_publisher_label', 'license_label', 'category_main_label',
       'category_name_parent_label', 'category_name_parent_main_label',
       'category_name_label'],
      dtype='object')

In [ ]:
 # 0.5029752173028096 lgb normal

In [33]:
x_train.head()

,doi_cites,pub_journals,pub_dois,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,q,quant,solv,stat,supr,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submitter_label,doi_cites_max_submitter_label,doi_cites_median_submitter_label,doi_cites_std_submitter_label,doi_cites_q10_submitter_label,doi_cites_q25_submitter_label,doi_cites_q75_submitter_label,doi_cites_mean_category_main_label,doi_cites_count_category_main_label,doi_cites_sum_category_main_label,doi_cites_min_category_main_label,doi_cites_max_category_main_label,doi_cites_median_category_main_label,doi_cites_std_category_main_label,doi_cites_q10_category_main_label,doi_cites_q25_category_main_label,doi_cites_q75_category_main_label,doi_cites_mean_update_ym,doi_cites_count_update_ym,doi_cites_sum

In [45]:
feature = x_train.copy()
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(50)


,importance
diff_doi_cites_doi_cites_mean_category_main_label,45452.756427
te_diff_license_label_cites_mean,9845.287023
diff_rate_doi_cites_doi_cites_mean_category_main_label,9263.424620
te_rate_license_label_cites_mean,3971.043035
diff_doi_cites_doi_cites_mean_doi_id_label,1151.217911
doi_cites,696.934825
diff_doi_cites_doi_cites_mean_update_ym,596.695166
last_created_ym,393.102887
diff_rate_doi_cites_doi_cites_mean_pub_publisher_label,359.453805
category_main_label,283.692540


In [14]:
df_feature.tail(400)


,importance
rbert_vec_raw_768_665,0.0
rbert_vec_raw_768_697,0.0
cs.ds,0.0
rbert_vec_raw_768_695,0.0
rbert_vec_raw_768_693,0.0
...,...
rbert_vec_raw_768_341,0.0
bayes-an,0.0
atom-ph,0.0
rbert_vec_raw_768_344,0.0
